[View in Colaboratory](https://colab.research.google.com/github/malinenimaurya/Deep-Learning/blob/master/transfer_learning_cifar_keras.ipynb)

In [38]:
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import to_categorical
from keras.datasets import cifar10
np.random.seed(10)

#Loading data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape)
print(x_test.shape)




(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [39]:
x_train.shape[0]

50000

In [40]:
#Normalising the data and dividing the dataset into two halves for training one model from scratch and on other halve to perform transfer learning

X1_train = []
X1_test = []
X2_train = []
X2_test = []
Y1_train = []
Y1_test = []
Y2_train = []
Y2_test = []

for i in range(x_train.shape[0]):
    if y_train[i] < 5:
        # put data in set 1
        X1_train.append(x_train[i]/255.0)
        Y1_train.append(y_train[i])
    else:
        # put data in set 2
        X2_train.append(x_train[i]/255.0)
        Y2_train.append(y_train[i])

for i in range(x_test.shape[0]):
    if y_test[i] < 5:
        # put data in set 1
        X1_test.append(x_test[i]/255.0)
        Y1_test.append(y_test[i])
    else:
        # put data in set 2
        X2_test.append(x_test[i]/255.0)
        Y2_test.append(y_test[i])

#Converting to numpy array

X1_train = np.asarray(X1_train).reshape((-1, 32, 32, 3))
X1_test = np.asarray(X1_test).reshape((-1, 32, 32, 3))
X2_train = np.asarray(X2_train).reshape((-1, 32, 32, 3))
X2_test = np.asarray(X2_test).reshape((-1, 32, 32, 3))

#one hot encoding
Y1_train = to_categorical(Y1_train)
Y1_test = to_categorical(Y1_test)

Y2_train = to_categorical(Y2_train)
Y2_test = to_categorical(Y2_test)

print (X1_train.shape, X1_test.shape)
print (Y1_train.shape, Y1_test.shape)
print (X2_train.shape, X2_test.shape)
print (Y2_train.shape, Y2_test.shape)

#Splitting into train and validation 
split1 = int(0.8 * X1_train.shape[0])
split2 = int(0.8 * X2_train.shape[0])

x1_val = X1_train[split1:]
x1_train = X1_train[:split1]
y1_val = Y1_train[split1:]
y1_train = Y1_train[:split1]

x2_val = X2_train[split2:]
x2_train = X2_train[:split2]
y2_val = Y2_train[split2:]
y2_train = Y2_train[:split2]






(25000, 32, 32, 3) (5000, 32, 32, 3)
(25000, 5) (5000, 5)
(25000, 32, 32, 3) (5000, 32, 32, 3)
(25000, 10) (5000, 10)


In [41]:

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu'))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(5, activation='softmax'))
#model architecture
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Measuring the run time of the model
import time, datetime

start = datetime.datetime.now()
hist1 = model.fit(x1_train, y1_train,
         epochs=10,
         shuffle=True,
         batch_size=100,
         validation_data=(x1_val, y1_val), verbose=2)

time_taken = datetime.datetime.now() - start
print ('\n'*2, '-'*20, '\n')
print ('Time taken for first training: ', time_taken)
print ('\n', '-'*20, '\n'*2)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 28, 28, 16)        4624      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 12, 12, 8)         1160      
_________________________________________________________________
flatten_4 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 1152)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               147584    
__________

In [46]:
# Final evaluation of the model
    scores = model.evaluate(X1_test, Y1_test, verbose=0)
    print("Accuracy: "+ str(scores[1]*100)+"%")


Accuracy: 72.86%


In [42]:
#Freezing the first 6 layers
for l in model.layers[:6]:
    l.trainable = False   



trans_model = Sequential(model.layers[:6])

trans_model.add(Dense(128, activation='relu'))
trans_model.add(Dropout(0.4))
trans_model.add(Dense(10, activation='softmax'))

trans_model.summary()
trans_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


start = datetime.datetime.now()
hist2 = trans_model.fit(x2_train, y2_train, epochs=10, shuffle=True, batch_size=100, validation_data=(x2_val, y2_val), verbose=2)
time_taken = datetime.datetime.now() - start
print ('\n'*2, '-'*20, '\n')
print ('Time taken for final training: ', time_taken)
print ('\n', '-'*20, '\n'*2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 28, 28, 16)        4624      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 12, 12, 8)         1160      
_________________________________________________________________
flatten_4 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 1152)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               147584    
__________

In [45]:
# Final evaluation of the model
    scores = trans_model.evaluate(X2_test, Y2_test, verbose=0)
    print("Accuracy: "+ str(scores[1]*100)+"%")


Accuracy: 79.08%
